Please submit the following as a Github repo:

1) Data Extraction and Transformation

One challenge  is correctly reporting international orders when sales are made in different currencies. 
To solve this, we pull reference exchange rate data that can be used to calculate and report international sales in different currencies. 
The European Central Bank provides an API to collect this data, **[documentation can be found here](https://sdw-wsrest.ecb.europa.eu/help/)** - the **data tab** should provide the information needed for the excercise.

**Using python**, write a script that will **pull exchange rate data that can be used for reporting**. For this exercise, 
we are looking for :
- data for February 9-10, 2023 
- using the Key Euro Area Indicators dataflow. 
Pull data for all daily currencies against the Euro.

Using fake ecommerce data using the **[API documented here](https://fakeapi.platzi.com/en/rest/introduction)** :
-  pull data for all products in the ‘Shoes’ category. 
- Assuming the data for these products is in USD, 
    - create a new pandas dataframe that includes all of the products
    - the price in USD, 
    - the price in EUR, 
    - the date of the exchange rate that was used. 

In [15]:
import requests     
import pandas as pd 
import io

In [16]:
# Building blocks for the URL
protocol =  'https://'
entrypoint = 'sdw-wsrest.ecb.europa.eu/service/' 
resource = 'data'           
flowRef ='EXR'              # Dataflow for exchange rates 
key = 'D..EUR.SP00.A'    # Defining the dimension values: Daily, all currency, to EURO rate, 

# Define the parameters
params = {
    'startPeriod': '2023-02-09',  # Start date of the time series
    'endPeriod': '2023-02-10',     # End of the time series
    #'format': 'csvdata'
}

In [17]:
# Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.USD.EUR.SP00.A
url = protocol + entrypoint + resource + '/'+ flowRef + '/' + key

# Make the HTTP request
response = requests.get(url, params=params, headers={'Accept': 'text/csv'})

# Check if the response returns succesfully with response code 200
print(response)

# Print the full URL
print(response.url)

<Response [200]>
https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D..EUR.SP00.A?startPeriod=2023-02-09&endPeriod=2023-02-10


In [18]:
# data to dataframe
df_exrEU = pd.read_csv(io.StringIO(response.text))
print(df_exrEU.columns)
df_exrEU.head()

Index(['KEY', 'FREQ', 'CURRENCY', 'CURRENCY_DENOM', 'EXR_TYPE', 'EXR_SUFFIX',
       'TIME_PERIOD', 'OBS_VALUE', 'OBS_STATUS', 'OBS_CONF', 'OBS_PRE_BREAK',
       'OBS_COM', 'TIME_FORMAT', 'BREAKS', 'COLLECTION', 'COMPILING_ORG',
       'DISS_ORG', 'DOM_SER_IDS', 'PUBL_ECB', 'PUBL_MU', 'PUBL_PUBLIC',
       'UNIT_INDEX_BASE', 'COMPILATION', 'COVERAGE', 'DECIMALS', 'NAT_TITLE',
       'SOURCE_AGENCY', 'SOURCE_PUB', 'TITLE', 'TITLE_COMPL', 'UNIT',
       'UNIT_MULT'],
      dtype='object')


,KEY,FREQ,CURRENCY,CURRENCY_DENOM,EXR_TYPE,EXR_SUFFIX,TIME_PERIOD,OBS_VALUE,OBS_STATUS,OBS_CONF,...,COMPILATION,COVERAGE,DECIMALS,NAT_TITLE,SOURCE_AGENCY,SOURCE_PUB,TITLE,TITLE_COMPL,UNIT,UNIT_MULT
0,EXR.D.AUD.EUR.SP00.A,D,AUD,EUR,SP00,A,2023-02-09,1.5409,A,F,...,NaN,NaN,4,NaN,4F0,NaN,Australian dollar/Euro,"ECB reference exchange rate, Australian dollar...",AUD,0
1,EXR.D.AUD.EUR.SP00.A,D,AUD,EUR,SP00,A,2023-02-10,1.5432,A,F,...,NaN,NaN,4,NaN,4F0,NaN,Australian dollar/Euro,"ECB reference exchange rate, Australian dollar...",AUD,0
2,EXR.D.BGN.EUR.SP00.A,D,BGN,EUR,SP00,A,2023-02-09,1.9558,A,F,...,NaN,NaN,4,NaN,4F0,NaN,Bulgarian lev/Euro,"ECB reference exchange rate, Bulgarian lev/Eur...",BGN,0
3,EXR.D.BGN.EUR.SP00.A,D,BGN,EUR,SP00,A,2023-02-10,1.9558,A,F,...,NaN,NaN,4,NaN,4F0,NaN,Bulgarian lev/Euro,"ECB reference exchange rate, Bulgarian lev/Eur...",BGN,0
4,EXR.D.BRL.EUR.SP00.A,D,BRL,EUR,SP00,A,2023-02-09,5.6165,A,F,...,NaN,NaN,4,NaN,4F0,NaN,Brazilian real/Euro,"ECB reference exchange rate, Brazilian real/Eu...",BRL,0


In [19]:
# select columns of interest, rename, lowercase (glue job), reorder for what makes sense to readability of stakeholders, sort by currency
df_exrEU = df_exrEU[['TIME_PERIOD','OBS_VALUE','CURRENCY']]
df_exrEU.rename({'OBS_VALUE':'exr_eur'},inplace=True, axis=1)
df_exrEU.columns= df_exrEU.columns.str.lower()
df_exrEU = df_exrEU[['time_period', 'currency', 'exr_eur']].sort_values('currency')
df_exrEU

,time_period,currency,exr_eur
0,2023-02-09,AUD,1.54090
1,2023-02-10,AUD,1.54320
2,2023-02-09,BGN,1.95580
3,2023-02-10,BGN,1.95580
4,2023-02-09,BRL,5.61650
5,2023-02-10,BRL,5.62450
6,2023-02-09,CAD,1.44230
7,2023-02-10,CAD,1.43640
8,2023-02-09,CHF,0.98900
9,2023-02-10,CHF,0.98720


In [20]:
### get shoes categoryid

url = "https://api.escuelajs.co/api/v1/categories"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code}")

### id 4


[{'id': 1, 'name': 'Clothes', 'image': 'https://api.lorem.space/image/fashion?w=640&h=480&r=2813', 'creationAt': '2023-03-11T05:52:10.000Z', 'updatedAt': '2023-03-11T05:52:10.000Z'}, {'id': 2, 'name': 'ele', 'image': 'https://api.lorem.space/image/watch?w=640&h=480&r=1345', 'creationAt': '2023-03-11T05:52:10.000Z', 'updatedAt': '2023-03-11T13:42:39.000Z'}, {'id': 3, 'name': 'Furniture', 'image': 'https://api.lorem.space/image/furniture?w=640&h=480&r=1249', 'creationAt': '2023-03-11T05:52:10.000Z', 'updatedAt': '2023-03-11T05:52:10.000Z'}, {'id': 4, 'name': 'Shoes', 'image': 'https://api.lorem.space/image/shoes?w=640&h=480&r=6370', 'creationAt': '2023-03-11T05:52:10.000Z', 'updatedAt': '2023-03-11T05:52:10.000Z'}, {'id': 5, 'name': 'kkkk', 'image': 'https://api.lorem.space/image?w=640&h=480&r=8085', 'creationAt': '2023-03-11T05:52:10.000Z', 'updatedAt': '2023-03-11T14:26:41.000Z'}, {'id': 6, 'name': 'sss', 'image': 'https://th.bing.com/th?id=OIP.GCw9SLDd9mlYzcqPUW8WAAHaJQ&w=223&h=279&c=

In [21]:
### get all products by categoryid

url = "https://api.escuelajs.co/api/v1/categories/4/products"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code}")


[{'id': 42, 'title': 'Bespoke Bronze Shirt', 'price': 808, 'description': 'Andy shoes are designed to keeping in mind durability as well as trends, the most stylish range of shoes & sandals', 'images': ['https://api.lorem.space/image/shoes?w=640&h=480&r=6149', 'https://api.lorem.space/image/shoes?w=640&h=480&r=1568', 'https://api.lorem.space/image/shoes?w=640&h=480&r=2424'], 'creationAt': '2023-03-11T05:52:10.000Z', 'updatedAt': '2023-03-11T05:52:10.000Z', 'category': {'id': 4, 'name': 'Shoes', 'image': 'https://api.lorem.space/image/shoes?w=640&h=480&r=6370', 'creationAt': '2023-03-11T05:52:10.000Z', 'updatedAt': '2023-03-11T05:52:10.000Z'}}, {'id': 51, 'title': 'asadsasdadasdasd', 'price': 100, 'description': 'The slim & simple Maple Gaming Keyboard from Dev Byte comes with a sleek body and 7- Color RGB LED Back-lighting for smart functionality', 'images': ['https://api.lorem.space/image/shoes?w=640&h=480&r=3589', 'https://api.lorem.space/image/shoes?w=640&h=480&r=2134', 'https://api

In [22]:
# data to dataframe
json_shoes = response.json()
df_shoes = pd.DataFrame(json_shoes)
print(df_shoes.columns)
print(df_shoes.shape)
df_shoes.head()

Index(['id', 'title', 'price', 'description', 'images', 'creationAt',
       'updatedAt', 'category'],
      dtype='object')
(24, 8)


,id,title,price,description,images,creationAt,updatedAt,category
0,42,Bespoke Bronze Shirt,808,Andy shoes are designed to keeping in mind dur...,[https://api.lorem.space/image/shoes?w=640&h=4...,2023-03-11T05:52:10.000Z,2023-03-11T05:52:10.000Z,"{'id': 4, 'name': 'Shoes', 'image': 'https://a..."
1,51,asadsasdadasdasd,100,The slim & simple Maple Gaming Keyboard from D...,[https://api.lorem.space/image/shoes?w=640&h=4...,2023-03-11T05:52:10.000Z,2023-03-11T14:27:27.000Z,"{'id': 4, 'name': 'Shoes', 'image': 'https://a..."
2,59,Bespoke Steel Chair,256,The Football Is Good For Training And Recreati...,[https://api.lorem.space/image/shoes?w=640&h=4...,2023-03-11T05:52:10.000Z,2023-03-11T05:52:10.000Z,"{'id': 4, 'name': 'Shoes', 'image': 'https://a..."
3,68,Practical Cotton Car,54,Ergonomic executive chair upholstered in bonde...,[https://api.lorem.space/image/shoes?w=640&h=4...,2023-03-11T05:52:10.000Z,2023-03-11T05:52:10.000Z,"{'id': 4, 'name': 'Shoes', 'image': 'https://a..."
4,81,Bespoke Rubber Keyboard,127,The automobile layout consists of a front-engi...,[https://api.lorem.space/image/shoes?w=640&h=4...,2023-03-11T05:52:10.000Z,2023-03-11T05:52:10.000Z,"{'id': 4, 'name': 'Shoes', 'image': 'https://a..."


In [23]:
## select cols of df_shoes 
df_shoes = df_shoes[['title','price']]
df_shoes['currency'] = 'USD'
print(df_shoes.columns)
print(df_shoes.shape)
df_shoes.head(100)

Index(['title', 'price', 'currency'], dtype='object')
(24, 3)


,title,price,currency
0,Bespoke Bronze Shirt,808,USD
1,asadsasdadasdasd,100,USD
2,Bespoke Steel Chair,256,USD
3,Practical Cotton Car,54,USD
4,Bespoke Rubber Keyboard,127,USD
5,Handmade Granite Salad,570,USD
6,Incredible Wooden Towels,740,USD
7,Oriental Fresh Shirt,453,USD
8,Ergonomic Cotton Car,539,USD
9,Elegant Fresh Soap,332,USD


In [24]:
## filter df_exrEU by US
df_EUUS = df_exrEU.query('currency == "USD"')
print(df_EUUS.columns)
print(df_EUUS.shape)
df_EUUS

Index(['time_period', 'currency', 'exr_eur'], dtype='object')
(2, 3)


,time_period,currency,exr_eur
56,2023-02-09,USD,1.0771
57,2023-02-10,USD,1.0690


In [25]:
## left join df_shoes to df_EUUS on currency
df_shoes_EUUS =  df_shoes.merge(df_EUUS, on='currency', how='left')
print(df_shoes_EUUS.columns)
print(df_shoes_EUUS.shape)
df_shoes_EUUS

Index(['title', 'price', 'currency', 'time_period', 'exr_eur'], dtype='object')
(48, 5)


,title,price,currency,time_period,exr_eur
0,Bespoke Bronze Shirt,808,USD,2023-02-09,1.0771
1,Bespoke Bronze Shirt,808,USD,2023-02-10,1.0690
2,asadsasdadasdasd,100,USD,2023-02-09,1.0771
3,asadsasdadasdasd,100,USD,2023-02-10,1.0690
4,Bespoke Steel Chair,256,USD,2023-02-09,1.0771
5,Bespoke Steel Chair,256,USD,2023-02-10,1.0690
6,Practical Cotton Car,54,USD,2023-02-09,1.0771
7,Practical Cotton Car,54,USD,2023-02-10,1.0690
8,Bespoke Rubber Keyboard,127,USD,2023-02-09,1.0771
9,Bespoke Rubber Keyboard,127,USD,2023-02-10,1.0690


In [26]:
'''
Using fake ecommerce data using the API documented here - 
pull data for all products in the ‘Shoes’ category. 
Assuming the data for these products is in USD, create a new pandas dataframe that includes 
- all of the products, 
- the price in USD, 
- the price in EUR, 
- the date of the exchange rate that was used. 
'''
df_shoes_EUUS['price_EUR'] = round(df_shoes_EUUS['price'] / df_shoes_EUUS['exr_eur'],2)
df_shoes_EUUS.rename({'title':'product_name', 'price':'price_US', 'time_period':'date_exr'}, axis=1, inplace=True) 
df_shoes_EUUS.drop(['exr_eur','currency'], axis=1, inplace=True)
df_shoes_EUUS = df_shoes_EUUS[['product_name', 'price_US', 'price_EUR', 'date_exr']]
df_shoes_EUUS

,product_name,price_US,price_EUR,date_exr
0,Bespoke Bronze Shirt,808,750.16,2023-02-09
1,Bespoke Bronze Shirt,808,755.85,2023-02-10
2,asadsasdadasdasd,100,92.84,2023-02-09
3,asadsasdadasdasd,100,93.55,2023-02-10
4,Bespoke Steel Chair,256,237.68,2023-02-09
5,Bespoke Steel Chair,256,239.48,2023-02-10
6,Practical Cotton Car,54,50.13,2023-02-09
7,Practical Cotton Car,54,50.51,2023-02-10
8,Bespoke Rubber Keyboard,127,117.91,2023-02-09
9,Bespoke Rubber Keyboard,127,118.80,2023-02-10


In [27]:
#### full dev code

# imports
import requests     
import pandas as pd 
import io

################### Connect to EBC API and get exchange rate data
# Building blocks for the URL
protocol =  'https://'
entrypoint = 'sdw-wsrest.ecb.europa.eu/service/' 
resource = 'data'           
flowRef ='EXR'              # Dataflow for exchange rates 
key = 'D..EUR.SP00.A'    # Defining the dimension values: Daily, all currency, to EURO rate 

# Define the parameters
params = {
    'startPeriod': '2023-02-09',
    'endPeriod': '2023-02-10',       
}

# Construct the URL
url = protocol + entrypoint + resource + '/'+ flowRef + '/' + key

# Make the HTTP request
response = requests.get(url, params=params, headers={'Accept': 'text/csv'})

# Check if the response returns succesfully with response code 200
print(response)

# Print the full URL
print(response.url)


# EBC data to dataframe
df_exrEU = pd.read_csv(io.StringIO(response.text))
print(df_exrEU.columns)
print(df_exrEU.shape)
print(df_exrEU.head(3))

# select columns , rename, lowercase (glue job), reorder for what makes sense to readability of stakeholders, sort by currency
df_exrEU = df_exrEU[['TIME_PERIOD','OBS_VALUE','CURRENCY']]
df_exrEU.rename({'OBS_VALUE':'exr_eur'},inplace=True, axis=1)
df_exrEU.columns= df_exrEU.columns.str.lower()
df_exrEU = df_exrEU[['time_period', 'currency', 'exr_eur']].sort_values('currency')
print(df_exrEU.columns)
print(df_exrEU.shape)
print(df_exrEU.head(3))

result1 = df_exrEU

####################  Connect to Fake ecommerce API and get all products by categoryid = 4 (Shoes)

url = "https://api.escuelajs.co/api/v1/categories/4/products"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code}")

# Fake Ecomm data to dataframe
json_shoes = response.json()
df_shoes = pd.DataFrame(json_shoes)
print(df_shoes.columns)
print(df_shoes.shape)
print(df_shoes.head(3))


## select cols, create currency col for join
df_shoes = df_shoes[['title','price']]
df_shoes['currency'] = 'USD'
print(df_shoes.columns)
print(df_shoes.shape)
print(df_shoes.head(3))

## filtrando df_exrEU por US
df_EUUS = df_exrEU.query('currency == "USD"')
print(df_EUUS.columns)
print(df_EUUS.shape)
print(df_EUUS.head(3))

#################### Fake ecomm data <Left Join> EBC data on currency
# left join
df_shoes_EUUS =  df_shoes.merge(df_EUUS, on='currency', how='left')
print(df_shoes_EUUS.columns)
print(df_shoes_EUUS.shape)
print(df_shoes_EUUS.head(3))

# calculate EUR price, rename cols, drop col, reorganize cols
df_shoes_EUUS['price_EUR'] = round(df_shoes_EUUS['price'] / df_shoes_EUUS['exr_eur'],2)
df_shoes_EUUS.rename({'title':'product_name', 'price':'price_US', 'time_period':'date_exr'}, axis=1, inplace=True) 
df_shoes_EUUS.drop(['exr_eur','currency'], axis=1, inplace=True)
df_shoes_EUUS = df_shoes_EUUS[['product_name', 'price_US', 'price_EUR', 'date_exr']]
print(df_shoes_EUUS.columns)
print(df_shoes_EUUS.shape)
print(df_shoes_EUUS.head(3))

result2 = df_shoes_EUUS

print('EBC EXR data fetch:', result1.head(10))
print('Final result data fetch:', result2.head(10))

<Response [200]>
https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D..EUR.SP00.A?startPeriod=2023-02-09&endPeriod=2023-02-10
Index(['KEY', 'FREQ', 'CURRENCY', 'CURRENCY_DENOM', 'EXR_TYPE', 'EXR_SUFFIX',
       'TIME_PERIOD', 'OBS_VALUE', 'OBS_STATUS', 'OBS_CONF', 'OBS_PRE_BREAK',
       'OBS_COM', 'TIME_FORMAT', 'BREAKS', 'COLLECTION', 'COMPILING_ORG',
       'DISS_ORG', 'DOM_SER_IDS', 'PUBL_ECB', 'PUBL_MU', 'PUBL_PUBLIC',
       'UNIT_INDEX_BASE', 'COMPILATION', 'COVERAGE', 'DECIMALS', 'NAT_TITLE',
       'SOURCE_AGENCY', 'SOURCE_PUB', 'TITLE', 'TITLE_COMPL', 'UNIT',
       'UNIT_MULT'],
      dtype='object')
(60, 32)
                    KEY FREQ CURRENCY CURRENCY_DENOM EXR_TYPE EXR_SUFFIX  \
0  EXR.D.AUD.EUR.SP00.A    D      AUD            EUR     SP00          A   
1  EXR.D.AUD.EUR.SP00.A    D      AUD            EUR     SP00          A   
2  EXR.D.BGN.EUR.SP00.A    D      BGN            EUR     SP00          A   

  TIME_PERIOD  OBS_VALUE OBS_STATUS OBS_CONF  ...  COMPILATION  COVE